# Create Heat Map for significant Fc gamma R-mediated phagocytosis genes

Pancancer heatmaps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.iloc[[7]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
7,Fc gamma R-mediated phagocytosis Homo sapiens ...,31/93,3.849187e-07,0.000014,0,0,2.5533,37.71284,WAS;PIK3CD;ASAP1;PIK3R1;PLD1;PIK3CG;PAK1;SCIN;...,KEGG_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 7
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

Fc gamma R-mediated phagocytosis Homo sapiens hsa04666
total genes: 31


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

31

In [6]:
# Only include p-values < a 
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [8]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [9]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [10]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MAPK3', 'SYK', 'PLD1', 'FCGR2A', 'PLCG2', 'MYO10', 'PIK3R1',
       'AKT2', 'SCIN', 'PAK1', 'AMPH'], dtype=object)

In [11]:
# Only include p-values < a 
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [13]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=500)

Loading BokehJS ...

# Sig in multiple cancers

In [14]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [17]:
enr2.res2d.iloc[[3]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
3,Fc gamma R-mediated phagocytosis Homo sapiens ...,12/93,2.188722e-08,0.000002,0,0,8.297894,146.352976,PAK1;PTPRC;SYK;PIK3CA;MYO10;PRKCB;INPP5D;RAC2;...,KEGG_2016


In [18]:
index2 = 3
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

Fc gamma R-mediated phagocytosis Homo sapiens hsa04666
total genes: 12


In [19]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['DOCK2', 'SYK', 'PRKCB', 'PLCG2', 'PIK3CD', 'INPP5D', 'MYO10',
       'PAK1', 'WAS', 'PTPRC', 'RAC2', 'PIK3CA'], dtype=object)

In [20]:
# Only include p-values < a 
a = 0.01
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [22]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=600)

Loading BokehJS ...

In [23]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['SYK', 'PLCG2', 'MYO10', 'PAK1'], dtype=object)

In [24]:
# Only include p-values < a 
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [26]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=350)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [24]:
#gnm = list(genes_pn_mult.Proteomics.unique())
gnm = genes_mult
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['MSH6', 'RFC3', 'PCNA', 'RFC4', 'MSH2', 'RFC2']


In [25]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Mismatch repair Homo sapiens hsa03430,6/23,1.135581e-18,3.327252e-16,0,0,869.565217,35929.901929,MSH6;RFC3;PCNA;RFC4;MSH2;RFC2,KEGG_2016
1,DNA replication Homo sapiens hsa03030,4/36,1.322041e-10,1.936791e-08,0,0,370.370370,8424.694035,RFC3;PCNA;RFC4;RFC2,KEGG_2016
2,Nucleotide excision repair Homo sapiens hsa03420,4/47,3.999723e-10,3.906396e-08,0,0,283.687943,6138.900971,RFC3;PCNA;RFC4;RFC2,KEGG_2016
3,Colorectal cancer Homo sapiens hsa05210,2/62,1.406958e-04,1.030596e-02,0,0,107.526882,953.646311,MSH6;MSH2,KEGG_2016
4,Pathways in cancer Homo sapiens hsa05200,2/397,5.591940e-03,3.276877e-01,0,0,16.792611,87.093686,MSH6;MSH2,KEGG_2016


In [23]:
# Look at p-values 
certain_gene = 'PRPF6'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
212,PRPF6,0.000050,0.391870,Gbm
11890,PRPF6,0.006819,0.140160,Hnscc
27513,PRPF6,0.359535,0.144750,Luad
37369,PRPF6,0.393073,0.107700,Lscc
47188,PRPF6,0.161801,0.068850,Brca
58301,PRPF6,0.338623,-0.006257,Ov
65628,PRPF6,0.000959,-0.193600,En
82250,PRPF6,0.803451,-0.063000,Colon
